# data.world
## D4D Hack Financial Disclosures

> #### Setup  

> Before running data.world notebooks for the first time, you'll need to:  
1. Install data.world's Python package, including optional `pandas` dependencies: 
```shell
pip install git+git://github.com/datadotworld/data.world-py.git#egg=project[PANDAS]
```
1. Obtain an API access token at https://data.world/settings/advanced
1. Store API access token using the `dw` command-line tool: 
```shell
dw configure
```

> Once your environment is set up, these steps do not need to be repeated for other data.world notebooks.

In [1]:
import datadotworld as dw
import csv
import pandas as pd

In [2]:
# Datasets are referenced by their path
dataset_key = 'rflprr/d-4-d-hack-financial-disclosures'

# Or simply by their URL
dataset_key = 'https://data.world/rflprr/d-4-d-hack-financial-disclosures'

In [3]:
# Load dataset (onto the local file system)
dataset_local = dw.load_dataset(dataset_key, force_update=True)  # cached under ~/.dw/cache

In [4]:
# See what is in it
dataset_local.describe()

{'homepage': 'https://data.world/rflprr/d-4-d-hack-financial-disclosures',
 'name': 'rflprr_d-4-d-hack-financial-disclosures',
 'resources': [{'format': 'csv',
   'name': 'endnotes',
   'path': 'data/endnotes.csv'},
  {'format': 'csv',
   'name': 'filer-s-employment-agreements-and-arrangements',
   'path': 'data/filer-s-employment-agreements-and-arrangements.csv'},
  {'format': 'csv',
   'name': 'filer-s-employment-assets-_-income-and-retirement-accounts',
   'path': 'data/filer-s-employment-assets-&-income-and-retirement-accounts.csv'},
  {'format': 'csv',
   'name': 'filer-s-positions-held-outside-united-states-government',
   'path': 'data/filer-s-positions-held-outside-united-states-government.csv'},
  {'format': 'csv',
   'name': 'filer-s-sources-of-compensation-exceeding-_5-000-in-a-year',
   'path': 'data/filer-s-sources-of-compensation-exceeding-$5-000-in-a-year.csv'},
  {'format': 'csv', 'name': 'liabilities', 'path': 'data/liabilities.csv'},
  {'format': 'csv', 'name': 'names

# Next steps

- Run `help()` to learn more ways to access and use your data. Try:
  - `help(dw.load_dataset)`
  - `help(dw.query)`
- Learn more at: https://github.com/datadotworld/data.world-py and https://docs.data.world

### Let's reshape the data and study relationships between filers, organizations, and disclosure types...

In [5]:
# create an empty array to hold the new, reshaped data

d = []

### From each file, we'll loop through the records and grab the name of the worker, the name of the organization or entity related to the disclosure, and the location if available

In [6]:
c = 0
for i in dataset_local.tables['filer-s-employment-agreements-and-arrangements']:
    file = i['file']
    disclosure_type = "employment"
    org = i['employer-or-party']
    location = i['city-state']
    c = c + 1
    d.append({'file': file, 'disclosure_type': disclosure_type, 'org': org, 'location': location})

print(str(c) + " records added from the employment table")
print(str(len(d)) + " records in the new dataset so far")

187 records added from the employment table
187 records in the new dataset so far


In [7]:
c = 0
for i in dataset_local.tables['filer-s-employment-assets-_-income-and-retirement-accounts']:
    file = i['file']
    disclosure_type = "assets"
    org = i['description']
    location = None
    c = c + 1
    d.append({'file': file, 'disclosure_type': disclosure_type, 'org': org, 'location': location})


print(str(c) + " records added from the assets table")
print(str(len(d)) + " records in the new dataset so far")

2050 records added from the assets table
2237 records in the new dataset so far


In [8]:
c = 0
for i in dataset_local.tables['filer-s-positions-held-outside-united-states-government']:
    file = i['file']
    disclosure_type = "positions"
    org = i['organization-name']
    location = i['city-state']
    c = c + 1
    d.append({'file': file, 'disclosure_type': disclosure_type, 'org': org, 'location': location})

print(str(c) + " records added from the positions table")
print(str(len(d)) + " records in the new dataset so far")

812 records added from the positions table
3049 records in the new dataset so far


In [9]:
c = 0
for i in dataset_local.tables['filer-s-sources-of-compensation-exceeding-_5-000-in-a-year']:
    file = i['file']
    disclosure_type = "compensations"
    org = i['source-name']
    location = i['city-state']
    c = c + 1
    d.append({'file': file, 'disclosure_type': disclosure_type, 'org': org, 'location': location})
    
print(str(c) + " records added from the compensations table")
print(str(len(d)) + " records in the new dataset so far")

798 records added from the compensations table
3847 records in the new dataset so far


In [10]:
c = 0
for i in dataset_local.tables['liabilities']:
    file = i['file']
    disclosure_type = "liabilities"
    org = i['creditor-name']
    location = None
    c = c + 1
    d.append({'file': file, 'disclosure_type': disclosure_type, 'org': org, 'location': location})

print(str(c) + " records added from the liabilities table")
print(str(len(d)) + " records in the new dataset so far")

155 records added from the liabilities table
4002 records in the new dataset so far


In [11]:
c = 0
for i in dataset_local.tables['other-assets-and-income']:
    file = i['file']
    disclosure_type = "other_assets"
    org = i['description']
    location = None
    c = c + 1
    d.append({'file': file, 'disclosure_type': disclosure_type, 'org': org, 'location': location})

print(str(c) + " records added from the other_assets table")
print(str(len(d)) + " records in the new dataset so far")

7485 records added from the other_assets table
11487 records in the new dataset so far


In [12]:
# let's preview a record to see how it looks

d[0]

{'disclosure_type': 'employment',
 'file': 'Acosta, Rene Alexander',
 'location': 'Miami, Florida',
 'org': 'U.S. Century Bank'}

### let's loop through each record in the new dataset and use the crosswalk to add in the clustered version of the organization's name (if it exists)

In [15]:
for i in d:
    for o in dataset_local.tables['org_crosswalk']:
        if i['org'] == o['source_org_value']:
            i['org_cluster'] = o['org_group']

### if we put our dataset into a pandas dataframe, we can browse its profile:

In [16]:
df = pd.DataFrame(d)

In [17]:
df.describe()

,disclosure_type,file,location,org,org_cluster
count,11487,11487,1797,11487,2080
unique,6,103,390,9203,975
top,other_assets,"Liddell, Chris","New York, New York",Republican National Committee,Republican National Committee
freq,7485,1707,417,46,64


## with clustering, we went from 9179 unique organization names to 975. Not bad!

In [18]:
# to write the new dataset to csv, run this command: 
df.to_csv(path_or_buf="./d.csv")